# Tam sayı dizileri ikili matris haline dönüştürmek

Elimizde bulunan bir sayı dizisini olduğu gibi bir sinir ağına gönderirsek bu çalışmaz öncelikli olarak bu listenin bir tensöre dönüştürülmesi gerekiyor

In [1]:
import numpy as np

In [8]:
from keras.datasets import imdb
(train_data,train_labels),(test_data,test_labels)=imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 8s 0us/step


In [2]:
def vectorize_sequences(sequences,dimension=10000):
    '''Verilen bir tam sayı dizisini ikili matris haline getirir'''
    # (len(sequences),dimensions) boyutlarında tüm elemanları sıfır olan bir matris oluşturuyoruz.
    results=np.zeros((len(sequences),dimension))
    for i,sequence in enumerate(sequences):
        results[i,sequence]=1. # results[i]'nin istenen indexlerini 1 yapar.
    return results

In [3]:
s=np.array([3,5])
vectorize_sequences(s,10)

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [9]:
x_train=vectorize_sequences(train_data) # Eğitim verisinin hazırlanması
x_test=vectorize_sequences(test_data) # Test verisinin hazırlanması

***Ağınızı İnşa Etmek***
Eğer ki girdiler vektör, etiketler ise skaler ise bu durumda stack ağ tipinde sıkı bağlı (dense) katmanlardanve aktivasyon fonksiyonu relu olan bir ağ en iyi tercih olacaktır.  <code>Dense(16,activation='relu')</code> Buradaki ***16*** parametresi ağımızda kaç tane gizli katman olacağını söyler. ***Gizli birim sayısı gösterim uzayının boyutudur*** Ne kadar fazla birim daha karmaşık bir ağ yapısıdır. Bu fazla katmanlar ağımızın daha detaylı öğrenmesini sağlayacaktır ama bu durum over fit durumuna neden olabilir. 

Dense katmanlarından oluşan stack bir ağda iki önemli mimari karar vardır.
* Kaç katman olacağı (Aşağıdaki örnekte: Üç katmandan oluşacak. Üçüncü katman skaler çıktı üretecek)
* Her katmanda kaç gizli birim olacağı (Aşağıdaki örnekte: 16 gizli birim)

Ara katmanların aktivasyon fonksiyonu olarak ***relu*** kullanılacak. Son katmanda ise ***sigmoid*** kullanılarak çıktıların olasılık değerlerine (0-1) dönüştürecek. 

In [6]:
from keras import models
from keras import layers

model=models.Sequential()
model.add(layers.Dense(16, activation="relu",input_shape=(10000,)))
model.add(layers.Dense(16,activation="relu"))
model.add(layers.Dense(1,activation="sigmoid"))

Using TensorFlow backend.


Şimdi ise bir ***loss*** fonksiyonu ve ***optimization*** algoritması seçme adımına geldik. Aşağıdaki örnekte istediğimiz şey bir ikili sınıflandırma problemi olduğu ve ağımızın çıktısı bir olasılık değeri olduğu için ***binary_crossentropy*** kullanmak mantıklı olacaktır. Bu tek seçenek değil aslında ***mean_squared_error*** da kullanılabilir.

***relu*** gibi bir aktivasyon fonksiyonu ise doğrusallığı kaldıran ağımızın daha iyi öğrenmesini sağlayan fonksiyondur. relu dışında ***prelu*** ve ***elu*** aktivasyon fonksiyonları da kullanılabilir. Optimizer olarak ***rmsprop*** kullanacağız.

In [7]:
model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["accuracy"])